<a href="https://colab.research.google.com/github/kkipngenokoech/agents/blob/main/crewAI_autoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Agent Collaboration**

> **Agenda**
> - **Part 1** — CrewAI: build a 3-agent sequential pipeline (Researcher → Analyst → Writer) for an EdTech startup pitch
> - **Part 2** — AutoGen: build a 4-agent dynamic research team that writes and peer-reviews a literature review

# Part 1: CrewAI

**CrewAI** is a framework for orchestrating teams of AI agents that collaborate on complex, multi-step tasks.

**Why not just use one big prompt?**
Single-prompt approaches struggle with long, multi-step workflows. Agents can specialize, call tools, retain memory, and hand off results — producing higher-quality outputs on complex tasks.

**Our use case:** We will build a 3-agent pipeline to research infrastructure challenges in Rwanda and produce an investor pitch for a fictional EdTech startup called *EduReach Smart-Box*.

## Environment Setup

In [ ]:
# Install crewAI and standard tools
!pip install -q -U crewai crewai-tools

In [ ]:
import os
from google.colab import userdata

# Load keys from Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

## ⚙️ CrewAI Process Types

In CrewAI, you define how work flows between agents by choosing a **process type** when building your `Crew`.

**Sequential (default)**: tasks run one after another, like an assembly line. The output of each task is passed as context to the next. This is what we use here: Researcher → Analyst → Writer.

**Hierarchical**: a manager agent (auto-created or defined by you) oversees the team. It assigns tasks dynamically, reviews outputs, and decides who works on what. More flexible, but less predictable and more expensive (extra LLM calls for the manager).

For our `EduReach Rwanda` pipeline, **sequential** is the right choice — the flow is linear, the roles are clear, and we want full control over the order of execution.

## Overview of an Agent

In the CrewAI framework, an `Agent` is an autonomous unit that can:

* Perform specific tasks
* Make decisions based on its role and goal
* Use tools to accomplish objectives
* Communicate and collaborate with other agents
* Maintain memory of interactions
* Delegate tasks when allowed

<Tip>
  Think of an agent as a specialized team member with specific skills,
  expertise, and responsibilities. For example, a `Researcher` agent might excel
  at gathering and analyzing information, while a `Writer` agent might be better
  at creating content.
</Tip>



## Agent Attributes (Required)

| Attribute                               | Parameter                | Type                                  | Description                                                                                              |
| :-------------------------------------- | :----------------------- | :------------------------------------ | :------------------------------------------------------------------------------------------------------- |
| **Role**                                | `role`                   | `str`                                 | Defines the agent's function and expertise within the crew.                                              |
| **Goal**                                | `goal`                   | `str`                                 | The individual objective that guides the agent's decision-making.                                        |
| **Backstory**                           | `backstory`              | `str`                                 | Provides context and personality to the agent, enriching interactions.

Read more here: [Agents - CrewAI](https://docs.crewai.com/en/concepts/agents)

### LOCAL KNOWLEDGE BASE

In [ ]:
# This simulates the "Research" phase by providing verified local data.
rwanda_ict_data = """
- Rwanda Basic Education Board (REB) aims for 1:1 device ratio.
- "Smart Classrooms" exist but 15-20% of rural schools lack stable grid power.
- Electricity cost: ~227 RWF/kWh for residential/small business.
- Major challenge: High cost of international bandwidth; localized caching is preferred.
- Language: Kinyarwanda is primary in lower primary, English from P4 onwards.
"""

### DEFINE AGENTS

Under the hood, CrewAI's `knowledge_sources` works exactly like a RAG pipeline:

1. **Ingestion**: your content (string, PDF, URL, etc.) gets chunked and embedded into a vector store
2. **Retrieval**: when the agent needs information, it queries the vector store semantically rather than reading the whole document
3. **Augmentation**: the retrieved chunks are injected into the agent's context before it generates a response

The difference is that CrewAI abstracts all of that away from you. In a traditional RAG setup you'd manually manage the embeddings, vector DB, and retrieval logic yourself. Here you just pass `knowledge_sources=[...]` and CrewAI handles the rest.

In [ ]:
rwanda_knowledge = StringKnowledgeSource(content=rwanda_ict_data)

In [ ]:
researcher = Agent(
    role='Lead Researcher (Rwanda Specialist)',
    goal='Synthesize ICT infrastructure data for school deployment.',
    backstory='A former consultant for the Ministry of ICT in Kigali. You know the rural landscape intimately.',
    knowledge_sources=[rwanda_knowledge],
    verbose=True,
    allow_delegation=False
)

analyst = Agent(
    role='Technical Cost Analyst',
    goal='Calculate a hardware-as-a-service budget.',
    backstory='Expert in low-power hardware. You focus on Raspberry Pi, ESP32, and solar-battery cycles.',
    verbose=True,
    allow_delegation=False
)

writer = Agent(
    role='Investment Pitch Specialist',
    goal='Create a 3-slide pitch for the "EduReach Smart-Box".',
    backstory='You bridge the gap between social impact and financial sustainability.',
    verbose=True,
    allow_delegation=False
)

In CrewAI, `allow_delegation=True` means the agent can delegate to **any other agent in the same `Crew`**.

So if our analyst gets stuck on a question about local school statistics, it can say "I need to ask the researcher" and CrewAI will route that sub-task to the researcher automatically — without you explicitly coding that handoff.

It works through a built-in tool called `Delegate work to coworker` that CrewAI gives each agent when delegation is enabled. The agent literally calls it like a tool, passing the coworker's role and the question.

The risk is **uncontrolled back-and-forth**. For example:
- Analyst delegates to Researcher
- Researcher delegates back to Analyst for clarification
- **This loops and burns tokens**

That's why for a linear pipeline (Researcher → Analyst → Writer) it's cleaner to disable it and let the **Task** definitions control the flow instead — each task explicitly receives output from the previous one via `context`. That's the more predictable CrewAI pattern.

### DEFINE TASKS

In [ ]:
task_research = Task(
    description="Identify the 3 biggest technical hurdles for offline servers in rural Rwanda.",
    expected_output="A prioritized list of 3 technical hurdles based on the provided data.",
    agent=researcher
)

task_analysis = Task(
    description="Based on the identified technical hurdles from the researcher, design a 'Smart-Box' kit (Raspberry Pi 5 + 128GB SD + Solar Panel). Calculate the total cost for 100 schools.",
    expected_output="A budget breakdown including a 'Cost per School' metric.",
    context=[task_research],
    agent=analyst
)

task_pitch = Task(
    description="Based on the technical hurdles and budget analysis, create a 3-slide pitch: 1. The Gap, 2. The Smart-Box Solution, 3. The Economics. Focus on the 'Offline' advantage.",
    expected_output="A 3-slide markdown pitch deck.",
    context=[task_research, task_analysis],
    agent=writer
)

### 🚀 Assembling the Crew

This is where everything comes together. The `Crew` object is the orchestrator — it takes our agents and tasks and manages the execution.

### How it works

1. **`agents`**: the list of agents available to the crew. Order doesn't strictly matter here since tasks are already assigned to specific agents.

2. **`tasks`**: the list of tasks in the order they should execute. Since we're using `Process.sequential`, CrewAI runs them top to bottom: Research → Analysis → Pitch.

3. **`process=Process.sequential`**: tells CrewAI to run tasks one at a time, passing each output as context to the next task that needs it. No manager, no dynamic routing — just a clean assembly line.

4. **`kickoff()`**: starts the pipeline. CrewAI hands `task_research` to the `researcher`, waits for output, then moves to `task_analysis` with the analyst, and finally hands everything to the `writer` to produce the pitch deck.

### The flow
```
researcher  →  task_research  →  output (3 hurdles)
                                        ↓
analyst     →  task_analysis  →  output (budget breakdown)
                                        ↓
writer      →  task_pitch     →  output (3-slide pitch deck)
```

In [ ]:
edu_crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[task_research, task_analysis, task_pitch],
    process=Process.sequential
)

### KICKING OFF

In [ ]:
results = edu_crew.kickoff()

CrewAI AMP includes a Visual Agent Builder that simplifies agent creation and configuration without writing code. Design your agents visually and test them in real-time. [https://app.crewai.com/studio/v2](https://app.crewai.com/studio/v2)

crew-studio-interface.avif

---

## Part 2: AutoGen

**AutoGen** (by Microsoft Research) takes a conversational approach to multi-agent systems. Instead of structured pipelines, agents send messages to each other in a **group chat**, and a selector decides who speaks next.

**Use case:** A 4-agent team that autonomously searches ArXiv, outlines, writes, and peer-reviews a literature review on multi-agent frameworks.

### CONFIGURATION

In [ ]:
# Install AutoGen (AgentChat and OpenAI extension)
!pip install -q -U autogen-agentchat autogen-ext[openai]

# Install arxiv
!pip install -q -U arxiv

In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat, RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

print("✅ Imports successful")

### Model Client Factory

We define a factory function that returns a fresh `OpenAIChatCompletionClient` for each agent. This is the AutoGen pattern — each agent gets its own client instance.

In [ ]:
def get_model_client():
    return OpenAIChatCompletionClient(model="gpt-4o")

### DEFINE TOOLS

In [ ]:
def arxiv_search(query: str, max_results: int = 2) -> list:
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    return results

arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

### DEFINE AGENTS

In [ ]:
# The Researcher
researcher = AssistantAgent(
    name="Lead_Researcher",
    model_client=get_model_client(),
    tools=[arxiv_search_tool],
    system_message="""You are a Data Gatherer.
    1. Search ArXiv for 3-5 papers.
    2. Provide ONLY the title, abstract, and authors.
    3. DO NOT analyze or write the report. Pass the raw data to the Strategist."""
)

# 2. The Strategist: The "Architect" who creates the blueprint.
strategist = AssistantAgent(
    name="Strategist",
    model_client=get_model_client(),
    system_message="""You are a Research Architect.
    Look at the raw data from the Researcher and create a THEMATIC OUTLINE.
    Group the papers by common trends (e.g., 'Efficiency' vs 'Safety').
    The Writer is waiting for your blueprint."""
)

# The Writer: The "Synthesizer"
writer = AssistantAgent(
    name="Writer",
    model_client=get_model_client(),
    system_message="""You are a Professional Writer.
    Using the Strategist's outline and the Researcher's data, write the full review.
    Include proper ArXiv citations. If you are missing data, ask the Researcher."""
)

# 4. The Critic: The "Quality Gatekeeper"
critic = AssistantAgent(
    name="Critic",
    model_client=get_model_client(),
    system_message="""You are a Peer Reviewer.
    Review the Writer's draft. If you find formatting errors or inconsistencies:
    1. DO NOT say 'TERMINATE'.
    2. List the specific errors and tell the Writer to 'Provide a Revised Draft'.
    Only say 'TERMINATE' once the Writer has fixed ALL previously mentioned issues."""
)

### THE DYNAMICS

In [ ]:
termination = TextMentionTermination("TERMINATE")

# SelectorGroupChat acts like a Project Manager choosing who speaks.
team1 = SelectorGroupChat(
    participants=[researcher, strategist, writer, critic],
    model_client=get_model_client(),
    termination_condition=termination,
    allow_repeated_speaker=True # Essential so Writer can speak twice (Draft 1 -> Draft 2)
)

# Or keep the same agents, but define a strict "relay" order
# Order: Researcher -> Strategist -> Writer -> Critic
team2 = RoundRobinGroupChat(
    participants=[researcher, strategist, writer, critic],
    max_turns=2
)

### EXECUTION

In [ ]:
async def run_research():
    await Console(
        team1.run_stream(task="Research and write a literature review on 'Multi-Agent Orchestration Frameworks'.")
    )

await run_research()